# Mini-Batch Gradient Descent (MBGD) from Scratch

## 1. Concept
**Mini-Batch Gradient Descent** is a variation of gradient descent where the model parameters are updated using a small random subset of data (a "batch") rather than the entire dataset (Batch GD) or a single sample (Stochastic GD).

### **Advantages:**
* **Faster than Batch GD:** Updates weights more frequently.
* **More Stable than SGD:** The gradient is an average of a batch, reducing the noise/variance seen in single-sample updates.
* **Vectorization:** Can utilize matrix operations efficiently (unlike SGD loops).



## 2. Implementation Steps
To implement Mini-Batch Gradient Descent from scratch, we follow this algorithm (as seen in the `MBGDRegressor` class):

1.  **Initialization:**
    * Initialize the **weights (coefficients)** to 1s (or small random numbers) and the **bias (intercept)** to 0.
    * Set hyperparameters: `learning_rate`, `epochs`, and `batch_size`.

2.  **Epoch Loop:**
    * Repeat the process for a fixed number of `epochs`. Each epoch represents one full pass through the dataset (statistically).

3.  **Batching & Shuffling:**
    * Calculate the number of batches needed: $\text{Total Samples} / \text{Batch Size}$.
    * Inside the epoch, loop through these batches.
    * **Crucial Step:** For every batch, select a **random subset** of indices from the training data. This randomization ensures the model doesn't learn patterns based on the order of data.

4.  **Prediction (Forward Pass):**
    * Calculate the predicted values ($\hat{y}$) for the **current batch only**:
        $$\hat{y}_{batch} = X_{batch} \cdot w + b$$

5.  **Gradient Calculation:**
    * Calculate the error derivatives based on the current batch's loss.
    * **Intercept Derivative:**
        $$\frac{\partial J}{\partial b} = -2 \cdot \text{mean}(y_{batch} - \hat{y}_{batch})$$
    * **Coefficient Derivative:**
        $$\frac{\partial J}{\partial w} = -2 \cdot X_{batch}^T \cdot (y_{batch} - \hat{y}_{batch})$$

6.  **Parameter Update:**
    * Update the weights and bias by moving them in the opposite direction of the gradient:
        $$b_{new} = b_{old} - \eta \cdot \frac{\partial J}{\partial b}$$
        $$w_{new} = w_{old} - \eta \cdot \frac{\partial J}{\partial w}$$
    *(Where $\eta$ is the learning rate)*

In [1]:
from sklearn.datasets import load_diabetes

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
X,y = load_diabetes(return_X_y=True)

In [3]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [5]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [6]:
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [7]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568968

## Custom Implementation of Mini-Batch GD

### Algorithm Steps:
1.  **Initialize** weights and bias (intercept).
2.  **Loop** for a given number of epochs.
3.  **Shuffle/Sample** a random batch of size $k$ from the training data.
4.  **Calculate Prediction** ($\hat{y}$) for this batch only.
5.  **Calculate Gradient** (loss derivative) for this batch.
    $$\frac{\partial J}{\partial w} = -\frac{2}{k} X_{batch}^T \cdot (y_{batch} - \hat{y}_{batch})$$
6.  **Update Parameters**:
    $$w_{new} = w_{old} - \eta \cdot \text{gradient}$$

In [8]:
import random

class MBGDRegressor:
    
    def __init__(self,batch_size,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            
            for j in range(int(X_train.shape[0]/self.batch_size)):
                
                idx = random.sample(range(X_train.shape[0]),self.batch_size)
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [9]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)

In [10]:
mbr.fit(X_train,y_train)

151.75499641213347 [  34.82074718 -138.80044313  446.9397174   299.81963285  -24.79973124
  -88.04205368 -196.14601041  116.44298013  403.70052256  112.90885534]


In [11]:
y_pred = mbr.predict(X_test)

In [12]:
r2_score(y_test,y_pred)

0.4536779325422383

In [13]:
from sklearn.linear_model import SGDRegressor

## Sklearn Implementation using `partial_fit`

Scikit-Learn's `SGDRegressor` is typically used for Stochastic Gradient Descent (batch size = 1). However, we can simulate **Mini-Batch Gradient Descent** using the `partial_fit` method.

* **`fit()`**: Retrains the model from scratch.
* **`partial_fit()`**: Updates the existing model weights with the new batch of data provided.

In the code below, we manually feed random batches to the model in a loop.

In [14]:
sgd = SGDRegressor(learning_rate='constant',eta0=0.1)

In [15]:
batch_size = 35

for i in range(100):
    
    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])

In [16]:
sgd.coef_

array([  63.5750963 ,  -80.31193426,  329.7039424 ,  248.2093418 ,
         21.34650899,  -34.23273866, -156.57400648,  117.92029294,
        329.14551122,  127.40326715])

In [17]:
sgd.intercept_

array([152.16091999])

In [18]:
y_pred = sgd.predict(X_test)

In [19]:
r2_score(y_test,y_pred)

0.4303240321645895